# Assignment 3 - SF Business Closures in 2020

**Goal:** Analyze effects of COVID on business closures in SF.

**Questions:** How does gentrification affect small businesses? How did COVID-19 play a role in the business closures within the past year?

**Hypothesis:** Small businesses can benefit from gentrified cities because the new people moving in can afford to pay higher prices (as opposed to chain stores). But, with COVID-19, the movement in the cities have significantly decreased. It might be difficult for niche small businesses to get customers. Additionally, the higher rent in these areas can play a role on why a business might close down. 

In [149]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import json
from IPython.display import display
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go

import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls

In [150]:
username = 'trangvan'
api_key = 'FnLgUNnm89wknjLxP7Mn'

### Foot Traffic - Foursquare

From AWS: This free data set contains indexed foot traffic to 19 categories of venues. The indexed data is broken out geographically, with included data for National, SF, NYC, LA, and Seattle. The data is normalized against U.S. Census data to remove age, gender and geographical bias. Data is provided daily from 02/19/2020.

Link: https://aws.amazon.com/marketplace/pp/prodview-cjhkgxpn6vcce?qid=1620202344723&sr=0-1&ref_=srh_res_product_title#offers

In [152]:
dma = pd.read_csv("foursquare/dma_june_26.csv")
dma.head()

,,Airports,Auto Dealerships,Auto Shops,Banks,Bars,Big Box Stores,Casual Dining,Clothing Stores,Convenience Stores,...,Pet Stores,Salons/Barbershops,Shopping Malls,Trails,Warehouse Stores,Wireless Carriers,Veterinarians,Unnamed: 30,Unnamed: 31,Unnamed: 32
0,2020-02-19,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,NaN,NaN,NaN
1,2020-02-20,99.308559,100.311490,99.991341,100.212073,100.172602,96.663937,99.464391,98.484903,98.663921,...,99.685989,99.098706,98.672303,100.214551,98.737988,99.096091,99.575027,NaN,NaN,NaN
2,2020-02-21,98.915234,100.627235,100.051771,99.543667,99.846706,94.814386,95.682777,97.021339,97.408742,...,99.054227,98.383068,96.485819,100.164078,98.289837,98.332008,99.341598,NaN,NaN,NaN
3,2020-02-22,98.812745,101.212953,100.418056,99.302714,100.062527,94.216787,95.303559,96.474193,97.261347,...,98.677300,98.697396,95.974031,100.584055,98.898297,98.147823,99.747368,NaN,NaN,NaN
4,2020-02-23,99.828860,101.321218,100.490327,99.327377,99.013396,94.132938,94.053563,95.300109,97.211902,...,98.179356,98.857799,94.738570,100.784907,99.580351,98.193703,99.773682,NaN,NaN,NaN


In [153]:
dma.columns

Index([' ', 'Airports', 'Auto Dealerships', 'Auto Shops', 'Banks', 'Bars',
       'Big Box Stores', 'Casual Dining', 'Clothing Stores',
       'Convenience Stores', 'Discount Stores', 'Drug Stores', 'Fast Food',
       'Furniture Stores', 'Grocery', 'Gas Stations', 'Gyms',
       'Hardware Stores', 'Hotels', 'Liquor Stores', 'Movie Theaters',
       'Nail Salons', 'Offices', 'Pet Stores', 'Salons/Barbershops',
       'Shopping Malls', 'Trails', 'Warehouse Stores', 'Wireless Carriers',
       'Veterinarians', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32'],
      dtype='object')

In [154]:
dma = dma.drop(['Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32'], axis = 1)
dma = dma.rename(columns= {' ':'Date'}).dropna()

dma.head(10)

,Date,Airports,Auto Dealerships,Auto Shops,Banks,Bars,Big Box Stores,Casual Dining,Clothing Stores,Convenience Stores,...,Movie Theaters,Nail Salons,Offices,Pet Stores,Salons/Barbershops,Shopping Malls,Trails,Warehouse Stores,Wireless Carriers,Veterinarians
0,2020-02-19,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
1,2020-02-20,99.308559,100.311490,99.991341,100.212073,100.172602,96.663937,99.464391,98.484903,98.663921,...,99.521232,98.826625,99.903644,99.685989,99.098706,98.672303,100.214551,98.737988,99.096091,99.575027
2,2020-02-21,98.915234,100.627235,100.051771,99.543667,99.846706,94.814386,95.682777,97.021339,97.408742,...,93.629522,98.173953,99.906637,99.054227,98.383068,96.485819,100.164078,98.289837,98.332008,99.341598
3,2020-02-22,98.812745,101.212953,100.418056,99.302714,100.062527,94.216787,95.303559,96.474193,97.261347,...,91.436338,98.087168,99.942381,98.677300,98.697396,95.974031,100.584055,98.898297,98.147823,99.747368
4,2020-02-23,99.828860,101.321218,100.490327,99.327377,99.013396,94.132938,94.053563,95.300109,97.211902,...,88.432241,98.095915,99.652426,98.179356,98.857799,94.738570,100.784907,99.580351,98.193703,99.773682
5,2020-02-24,99.145877,99.982691,99.796841,105.740171,98.421520,92.673965,91.823398,91.449044,96.948163,...,84.273219,97.651470,103.881314,96.088754,98.363856,91.429502,97.404048,97.532149,97.953212,98.967531
6,2020-02-25,97.511398,99.667140,99.600311,103.094937,98.644360,93.004916,92.669926,91.500182,96.941470,...,83.862910,97.493982,103.437668,96.171267,98.268209,91.223484,96.897560,98.005495,97.972496,98.803162
7,2020-02-26,97.347270,99.753031,99.866883,103.090012,98.409616,93.501667,93.020188,91.670098,97.063267,...,83.965105,97.930199,102.997828,96.223462,98.310040,91.333485,96.259480,98.915619,98.901187,98.977683
8,2020-02-27,97.543759,100.440633,100.811910,103.612865,98.480321,94.281847,93.412206,92.519036,97.538527,...,83.868436,98.607867,102.560475,96.811208,98.968191,91.801666,96.461103,100.491101,100.312114,99.582156
9,2020-02-28,97.809226,100.930123,101.590712,106.338901,98.619361,94.974483,93.635338,92.742979,98.140527,...,83.033713,99.665074,102.610765,97.503550,99.733803,92.119356,96.573259,102.212258,102.310415,100.286332


In [170]:
def get_stats(category):
    cat_track = dma[category]

    print("{} Basic Stats:".format(category))
    cat_mean = np.mean(cat_track)
    cat_median = np.median(cat_track)
    cat_min = min(cat_track)
    cat_max = max(cat_track)
    
    min_idx = dma[category].where(dma[category]==cat_min).dropna().index
    min_row = dma.loc[min_idx]
    min_day = min_row['Date'].values[0]
    
    max_idx = dma[category].where(dma[category]==cat_max).dropna().index
    max_row = dma.loc[max_idx]
    max_day = max_row['Date'].values[0]
    
    print("Mean:", cat_mean)
    print("Median:", cat_median)
    print("Min:", cat_min, "on", min_day)
    print("Max:", cat_max, "on", max_day)
    
    return {"Mean": cat_mean, 
            "Median": cat_median, 
            "Min": [min_idx, min_day, cat_min],
            "Max": [max_idx, max_day, cat_max]}

In [171]:
airports_track = get_stats('Airports')

Airports Basic Stats:
Mean: 42.294292089224804
Median: 28.07891607
Min: 19.80723258 on 2020-04-18
Max: 100.0 on 2020-02-19


In [172]:
#Plotting all categories on the day of Airport's lowest foot tracking
airport_min_row = dma.loc[airports_track['Min'][0]]
flat_values = [i[0] for i in airport_min_row.values.T[1:]]    #Had to turn into a list of values
categories = list(airport_min_row.columns[1:])                #numpy array not valid, turn into list
fig = px.bar(airport_min_row, x= categories, y=flat_values)
fig.update_layout(title_text='Foot Tracking (%) on {}'.format(airports_track['Min'][1]), xaxis_type='category')
fig.update_xaxes(title_text = 'Category')
fig.update_yaxes(title_text = 'Percent(%)')
fig.update_traces(marker_color='#88BB92')
fig.show()

fig.write_html("out/airport_foot_tracking.html")
py.plot(fig, filename='airport_foot_tracking', auto_open=False)

'https://plotly.com/~trangvan/5/'

In [173]:
grocery_track = get_stats("Grocery")

Grocery Basic Stats:
Mean: 102.77026628503876
Median: 100.0
Min: 93.82419048 on 2020-06-14
Max: 135.1755095 on 2020-03-19


In [195]:
grocery_max_row = dma.loc[grocery_track['Max'][0]]
flat_values = [i[0] for i in grocery_max_row.values.T[1:]]    #Had to turn into a list of values
categories = list(grocery_max_row.columns[1:])                #numpy array not valid, turn into list
fig2 = px.bar(grocery_max_row, x= categories, y=flat_values)
fig2.update_layout(title_text='Foot Tracking (%) on {}'.format(grocery_track['Max'][1]), xaxis_type='category')
fig2.update_xaxes(title_text = 'Category')
fig2.update_yaxes(title_text = 'Percent(%)')
fig2.update_traces(marker_color='#FBA037')
fig2.update_layout(
    font=dict(
        family="Barlow",
        size=12
    )
)
fig2.show()

fig2.write_html("out/grocery_foot_tracking.html")
py.plot(fig2, filename='grocery_foot_tracking', auto_open=False)

'https://plotly.com/~trangvan/8/'

In [189]:
#Plot multiple categories - method from here: https://stackoverflow.com/questions/55822042/how-to-draw-a-multiple-line-chart-using-plotly-express
#df_melt = dma.melt(id_vars='Date', value_vars= list(dma.columns[1:]))
dma.columns

df_melt = dma.melt(id_vars='Date', value_vars=list(dma.columns)[1:])

df_melt = df_melt.rename(columns= {'variable': 'Industry'})
fig3 = px.line(df_melt, x='Date' , y='value' , color='Industry')
fig3.update_layout(title_text='Foot Tracking (%) From February 2020 to June 2020')
fig3.update_xaxes(title_text = 'Date')
fig3.update_yaxes(title_text = 'Percent(%)')
fig3.update_layout(
    font=dict(
        family="Barlow",
    )
)
fig3.show()

fig3.write_html("out/all_industries.html")
py.plot(fig3, filename='all_industries', auto_open=False)

'https://plotly.com/~trangvan/13/'

### DataSF

This dataset includes the locations of businesses that pay taxes to the City and County of San Francisco. Each registered business may have multiple locations and each location is a single row. The Treasurer & Tax Collector’s Office collects this data through business registration applications, account update/closure forms, and taxpayer filings. The data is collected to help enforce the Business and Tax Regulations Code including, but not limited to: Article 6, Article 12, Article 12-A, and Article 12-A-1.

Link: https://data.sfgov.org/Economy-and-Community/Registered-Business-Locations-San-Francisco/g8m3-pdis

In [88]:
reg_bus = pd.read_csv('datasf/Registered_Business_Locations_-_San_Francisco.csv')
reg_bus.head()

,Location Id,Business Account Number,Ownership Name,DBA Name,Street Address,City,State,Source Zipcode,Business Start Date,Business End Date,...,Supervisor District,Neighborhoods - Analysis Boundaries,Business Corridor,Business Location,UniqueID,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,Neighborhoods
0,0019621-01-001,19621,Dr Monetta Od A Prof Corp,Dr Monetta Od A Prof Corp,2532 Ocean Ave,San Francisco,CA,94132,10/01/1968,NaN,...,7.0,West of Twin Peaks,NaN,POINT (-122.47252 37.731396),0019621-01-001-0019621-6100-6299-10-01-1968,62.0,10.0,8.0,41.0,62.0
1,1276529-04-211,21715,Paige Structural Glass Co,Paige Glass Company,75 Williams Ave,San Francisco,CA,94124,10/01/1968,NaN,...,10.0,Bayview Hunters Point,NaN,POINT (-122.39465 37.729523),1276529-04-211-0021715-2300-2399-01-01-2021,86.0,2.0,9.0,1.0,86.0
2,0024471-01-001,24471,Darling-Delaware Co Inc,Darling International Inc,429 Amador St,San Francisco,CA,94124,10/01/1968,NaN,...,10.0,Bayview Hunters Point,NaN,POINT (-122.381645 37.745956),0024471-01-001-0024471-8100-8139 3100-3399-10-...,56.0,2.0,9.0,1.0,56.0
3,0189644-02-001,189644,Neirro Erainya Rodgers G Etal,239-245 Bartlett St Condo Assn,239 Bartlett St #245,San Francisco,CA,94110,01/01/1989,NaN,...,9.0,Mission,NaN,POINT (-122.41961 37.75313),0189644-02-001-0189644--01-24-2005,53.0,3.0,2.0,20.0,53.0
4,0034153-03-001,34153,Mccormick Thomas E,528 27th St Apts,528 27th St,San Francisco,CA,94131,10/01/1968,NaN,...,8.0,Noe Valley,NaN,POINT (-122.43243 37.74664),0034153-03-001-0034153-7210-7219-02-01-1970,84.0,9.0,5.0,22.0,84.0


In [95]:
bus_end = reg_bus[reg_bus['Location End Date'].notna()]
bus_end['Location End Date'] = pd.to_datetime(bus_end['Location End Date'])
bus_end['Location Start Date'] = pd.to_datetime(bus_end['Location Start Date'])

C:\Users\Trang\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Trang\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [96]:
bus_end['Location End Date']

8        2013-12-31
14       2020-12-31
18       2017-04-13
24       2019-12-31
25       2020-12-31
            ...    
272419   2020-11-29
272472   2021-03-31
272497   2021-02-16
272546   2021-03-26
272553   2021-03-15
Name: Location End Date, Length: 143429, dtype: datetime64[ns]

In [97]:
mask = (bus_end['Location End Date'] >= '2020-01-01') & (bus_end['Location End Date'] <= '2020-12-31')
end_2020 = bus_end.loc[mask]
end_2020.sort_values(by='Location End Date').head()

,Location Id,Business Account Number,Ownership Name,DBA Name,Street Address,City,State,Source Zipcode,Business Start Date,Business End Date,...,Supervisor District,Neighborhoods - Analysis Boundaries,Business Corridor,Business Location,UniqueID,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,Neighborhoods
146880,1245746-02-201,1111641,Thibault Duchemin,Ava,1355 Market St 488,San Francisco,CA,94103,01/01/2020,01/01/2020,...,6.0,South of Market,NaN,POINT (-122.416885 37.77692),1245746-02-201-1111641-5400-5499-01-01-2020,32.0,5.0,10.0,34.0,32.0
11643,0384616-01-001,384616,Murphy Kevin B,Franchise Foundations,4157 23rd St,San Francisco,CA,94114,01/24/2005,01/01/2020,...,8.0,Noe Valley,NaN,POINT (-122.435486 37.75279),0384616-01-001-0384616--01-24-2005,84.0,3.0,5.0,22.0,84.0
253876,1214934-03-191,1099133,All Around Access Llc,All Around Access Llc,2095 47th Ave,Naples,FL,34120,03/04/2019,01/01/2020,...,NaN,NaN,NaN,POINT (-81.56652 26.318302),1214934-03-191-1099133-4200-4299 4400-4599-03-...,NaN,NaN,NaN,NaN,NaN
19357,1066673-05-161,1032455,Osborne Clarke Llp,Osborne Clarke Llp,535 Mission St,San Francisco,CA,94105,11/01/2015,NaN,...,6.0,Financial District/South Beach,NaN,POINT (-122.39843 37.789047),1066673-05-161-1032455-5400-5499-11-01-2015,32.0,1.0,10.0,8.0,32.0
200212,0484059-01-001,484059,Colborn Danielle C,"Danielle Colborn, Phd Psycholo",Po Box 15178,San+francisco,CA,94115,10/11/2013,01/01/2020,...,5.0,Japantown,NaN,POINT (-122.43513 37.784897),0484059-01-001-0484059-6100-6299-10-11-2013,103.0,4.0,11.0,15.0,103.0


In [117]:
by_neighborhood = end_2020.groupby(by='Neighborhoods - Analysis Boundaries').size().to_frame('count').reset_index()
by_neighborhood

,Neighborhoods - Analysis Boundaries,count
0,Bayview Hunters Point,339
1,Bernal Heights,177
2,Castro/Upper Market,299
3,Chinatown,329
4,Excelsior,119
5,Financial District/South Beach,1668
6,Glen Park,47
7,Haight Ashbury,155
8,Hayes Valley,238
9,Inner Richmond,181


In [130]:
by_neighborhood['Neighborhoods - Analysis Boundaries']

0              Bayview Hunters Point
1                     Bernal Heights
2                Castro/Upper Market
3                          Chinatown
4                          Excelsior
5     Financial District/South Beach
6                          Glen Park
7                     Haight Ashbury
8                       Hayes Valley
9                     Inner Richmond
10                      Inner Sunset
11                         Japantown
12                         Lakeshore
13                      Lincoln Park
14                 Lone Mountain/USF
15                            Marina
16                           Mission
17                       Mission Bay
18                          Nob Hill
19                        Noe Valley
20                       North Beach
21        Oceanview/Merced/Ingleside
22                     Outer Mission
23                    Outer Richmond
24                   Pacific Heights
25                           Portola
26                      Potrero Hill
2

In [187]:
neighborhood_plt = px.bar(by_neighborhood, x='count', y='Neighborhoods - Analysis Boundaries', orientation='h', width=1000, height=800)
neighborhood_plt.update_layout(title_text='Business Closures By Neighborhood', yaxis={'categoryorder':'category descending'})
neighborhood_plt.update_traces(marker_color='#FFBFA0')
neighborhood_plt.update_layout(
    title="Business Closures By Neighborhood",
    xaxis_title="Number of Business Closed",
    yaxis_title="Neighborhood",
    font=dict(
        family="Barlow",
    )
)

neighborhood_plt.show()

neighborhood_plt.write_html("out/close_by_neighborhood.html")
py.plot(neighborhood_plt, filename='close_by_neighborhood', auto_open=False)

'https://plotly.com/~trangvan/15/'

In [78]:
end_2020['NAICS Code Description']

14             Real Estate and Rental and Leasing Services
25        Professional, Scientific, and Technical Services
35        Professional, Scientific, and Technical Services
76        Professional, Scientific, and Technical Services
81                                          Accommodations
                                ...                       
272160                                       Food Services
272299                                         Information
272388                                           Insurance
272397                                    Certain Services
272413                                            Multiple
Name: NAICS Code Description, Length: 6615, dtype: object

In [146]:
by_naics = end_2020.groupby(by='NAICS Code Description').size().to_frame('count').reset_index()
by_naics

,NAICS Code Description,count
0,Accommodations,667
1,Administrative and Support Services,209
2,"Arts, Entertainment, and Recreation",458
3,Certain Services,266
4,Construction,539
5,Financial Services,314
6,Food Services,842
7,Information,293
8,Insurance,37
9,Manufacturing,125


In [205]:
naics_pie = px.pie(by_naics, values='count', names='NAICS Code Description',hover_name="NAICS Code Description", hover_data={'count':True, 'NAICS Code Description': False})
naics_pie.update_layout(
    title="Business Closures By Industry (NAICS)",
    legend_title="Industry",
    font=dict(
        family="Barlow",
        size=12,
    )
)
naics_pie.show()
naics_pie.write_html("out/close_by_naics.html")
py.plot(naics_pie, filename='close_by_naics', auto_open=False)

'https://plotly.com/~trangvan/29/'

## Safegraph



In [3]:
! pip install -q --upgrade git+https://github.com/SafeGraphInc/safegraph_py

In [11]:
from safegraph_py_functions import safegraph_py_functions as sgpy
# sgpy.test_me() # returns 'Hello World' to ensure you have downloaded the library
# sgpy.help() # returns a list of all active functions and their arguments in the safegraph_py library

In [9]:
single_file_path = "safegraph/weekly_places/main-file/2020-06-08-weekly-patterns.csv.gz"
single_pattern_test = sgpy.read_pattern_single(single_file_path)

In [32]:
sf_places = single_pattern_test[single_pattern_test['city'] == 'San Francisco']

In [34]:
sf_places.to_csv('sf_data.csv')

### Social Distancing Metrics

This product is delivered daily (3 days delayed from actual). Daily data is available going back to January 1, 2019. We used v2.1 to create the historical data from Jan 1, 2019 - Dec 31, 2019 (the backfill) as well as the data from May 10, 2020 forwards. However, the Jan 1-May 9, 2020 data is on v2.0. Apologies for any inconvenience, please see Release Notes below for more information.

The data was generated using a panel of GPS pings from anonymous mobile devices. We determine the common nighttime location of each mobile device over a 6 week period to a Geohash-7 granularity (~153m x ~153m). For ease of reference, we call this common nighttime location, the device's "home". We then aggregate the devices by home census block group and provide the metrics set out below for each census block group.


Documentation: https://docs.safegraph.com/docs/social-distancing-metrics

In [25]:
# Social Distance Data

# loop through month and concat string "0" + "month" and loop through days => extract data
filename = "safegraph/social_distance/2020/{0}/{1}/2020-{0}-{1}-social-distancing.csv.gz".format('01','02')
social_distance = pd.read_csv(filename, nrows=100, compression='gzip', error_bad_lines=False)
social_distance.head()

,origin_census_block_group,date_range_start,date_range_end,device_count,distance_traveled_from_home,bucketed_distance_traveled,median_dwell_at_bucketed_distance_traveled,completely_home_device_count,median_home_dwell_time,bucketed_home_dwell_time,at_home_by_each_hour,part_time_work_behavior_devices,full_time_work_behavior_devices,destination_cbgs,delivery_behavior_devices,median_non_home_dwell_time,candidate_device_count,bucketed_away_from_home_time,median_percentage_time_home,bucketed_percentage_time_home
0,10139533002,2020-01-02T00:00:00-06:00,2020-01-03T00:00:00-06:00,56,21298,"{""16001-50000"":18,""0"":11,"">50000"":13,""2001-800...","{""16001-50000"":39,"">50000"":42,""<1000"":41,""2001...",12,831,"{""721-1080"":16,""361-720"":7,""61-360"":4,""<60"":10...","[43,41,42,41,38,41,33,29,27,22,18,17,18,18,23,...",5,6,"{""011010027002"":1,""011010056091"":1,""0101395320...",4,163,86,"{""21-45"":1,""481-540"":5,""541-600"":2,""46-60"":4,""...",82,"{""26-50"":2,""51-75"":12,""0-25"":8,"">100"":2,""76-10..."
1,10210601014,2020-01-02T00:00:00-06:00,2020-01-03T00:00:00-06:00,169,7623,"{""16001-50000"":17,""0"":33,"">50000"":39,""2001-800...","{""16001-50000"":172,"">50000"":49,""<1000"":184,""20...",32,833,"{""721-1080"":44,""361-720"":26,""61-360"":10,""<60"":...","[116,116,118,116,113,109,105,95,93,73,68,67,72...",28,14,"{""010479566002"":1,""010730108041"":1,""0111703031...",2,206,270,"{""21-45"":14,""481-540"":8,""541-600"":8,""46-60"":3,...",76,"{""26-50"":10,""51-75"":40,""0-25"":30,"">100"":1,""76-..."
2,10950303005,2020-01-02T00:00:00-06:00,2020-01-03T00:00:00-06:00,92,17011,"{""16001-50000"":25,""0"":21,"">50000"":11,""2001-800...","{""16001-50000"":56,"">50000"":43,""<1000"":49,""2001...",18,692,"{""721-1080"":16,""361-720"":11,""61-360"":10,""<60"":...","[52,52,51,50,53,50,47,43,39,33,31,33,36,23,27,...",8,6,"{""010950303004"":3,""010890025011"":1,""0108900280...",4,213,149,"{""21-45"":5,""481-540"":8,""541-600"":2,""46-60"":7,""...",64,"{""26-50"":6,""51-75"":21,""0-25"":22,"">100"":1,""76-1..."
3,11010055032,2020-01-02T00:00:00-06:00,2020-01-03T00:00:00-06:00,71,7667,"{""16001-50000"":4,""0"":20,"">50000"":2,""2001-8000""...","{""16001-50000"":82,"">50000"":103,""<1000"":96,""200...",23,880,"{""721-1080"":22,""361-720"":7,""61-360"":7,""<60"":8,...","[49,49,50,48,48,47,47,45,44,40,35,42,35,36,35,...",12,3,"{""011010053021"":1,""011010027002"":4,""0100301140...",3,181,147,"{""21-45"":1,""481-540"":4,""541-600"":1,""46-60"":1,""...",77,"{""26-50"":8,""51-75"":17,""0-25"":11,"">100"":3,""76-1..."
4,40210002061,2020-01-02T00:00:00-07:00,2020-01-03T00:00:00-07:00,495,15672,"{""16001-50000"":144,""0"":127,"">50000"":38,""2001-8...","{""16001-50000"":57,"">50000"":49,""<1000"":24,""2001...",127,842,"{""721-1080"":115,""361-720"":88,""61-360"":39,""<60""...","[346,345,348,348,343,338,326,299,271,241,226,2...",49,22,"{""040210002053"":1,""040210002041"":27,""040138163...",18,139,822,"{""21-45"":20,""481-540"":24,""541-600"":18,""46-60"":...",85,"{""26-50"":28,""51-75"":111,""0-25"":63,"">100"":5,""76..."


In [26]:
social_distance.columns

Index(['origin_census_block_group', 'date_range_start', 'date_range_end',
       'device_count', 'distance_traveled_from_home',
       'bucketed_distance_traveled',
       'median_dwell_at_bucketed_distance_traveled',
       'completely_home_device_count', 'median_home_dwell_time',
       'bucketed_home_dwell_time', 'at_home_by_each_hour',
       'part_time_work_behavior_devices', 'full_time_work_behavior_devices',
       'destination_cbgs', 'delivery_behavior_devices',
       'median_non_home_dwell_time', 'candidate_device_count',
       'bucketed_away_from_home_time', 'median_percentage_time_home',
       'bucketed_percentage_time_home'],
      dtype='object')